In [1]:
from transformer_lens import HookedTransformer

In [2]:
model = HookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
prompt = "I'm excited because I'm going to a "

In [24]:
control_prompt = "sad"

In [25]:
tokens = model.to_tokens(prompt)

In [26]:
control_tokens = model.to_tokens(control_prompt)

In [27]:
control_tokens

tensor([[50256,    82,   324]])

In [28]:
act_name = 11

In [29]:
coeff = 1.0

In [30]:
from transformer_lens import utils

In [31]:
hook_name = utils.get_act_name("mlp_out", 2)

In [32]:
hook_name

'blocks.2.hook_mlp_out'

In [33]:
_, control_activations = model.run_with_cache(control_tokens)

In [34]:
target_activations = control_activations[hook_name]

In [35]:
from IPython.core.debugger import set_trace

In [36]:
def add_activations(activations, hook):
    n_tokens = target_activations.shape[1]
    activations[:, :n_tokens, :] += target_activations * 10
    return activations

In [37]:
n_new_tokens = 20

In [38]:
generated_tokens = tokens

In [39]:
import torch.nn.functional as F
from einops import rearrange

In [40]:
import torch

In [41]:
def logit_to_pred_token(logits):
    pred_logits = logits[0, -1, :]
    probs = F.softmax(pred_logits, dim=-1)
    pred_token = torch.argmax(pred_logits, dim=-1)
    return pred_token

In [42]:
for _ in range(n_new_tokens):
    modified_logits = model.run_with_hooks(
        generated_tokens,
        fwd_hooks=[(hook_name, add_activations)]
    )
    pred_token = logit_to_pred_token(modified_logits)
    # set_trace()
    generated_tokens = torch.cat([
        generated_tokens,
        # rearrange(pred_token, "n -> 1 1 n")
        pred_token.unsqueeze(0).unsqueeze(0)
    ], dim=-1)
    
    print(model.to_string(generated_tokens))

["<|endoftext|>I'm excited because I'm going to a \xa0"]
["<|endoftext|>I'm excited because I'm going to a \xa0party"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I'm"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I'm going"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I'm going to"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I'm going to have"]
["<|endoftext|>I'm excited because I'm going to a \xa0party with my friends and I'm going to have a"]
["<|endoftext|>I'm excite